In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df=pd.read_excel('/content/drive/MyDrive/Compile_Label_Data.xlsx')
df.head()

,Lemma,Label
0,Video show Nancy Pelosi say give shoot kill ...,1
1,George Soros ban Russia Pakistan Turkey Hung...,1
2,U N call decriminalize sex minor,1
3,Scientists struggle understand Antarctica wa...,1
4,La Reserva Federal present su nuevo sistema ...,1


In [3]:
import re
import string
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [4]:
df["Lemma"] = df["Lemma"].apply(wordopt)

In [5]:
df["Lemma"].isnull().sum()

0

In [6]:
df['Label'].isnull().sum()

0

In [7]:
df.shape

(127420, 2)

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
train_data, val_data = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['Lemma'])
X_train_seq = tokenizer.texts_to_sequences(train_data['Lemma'])
X_val_seq = tokenizer.texts_to_sequences(val_data['Lemma'])

In [ ]:
max_seq_length = 1000
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_seq_length, padding='post', truncating='post')

In [ ]:
X_train_pad

array([[  28,    2, 1373, ...,    0,    0,    0],
       [  67,   48,  148, ...,    0,    0,    0],
       [ 125,  171,  479, ...,    0,    0,    0],
       ...,
       [2001,   84,  462, ...,    0,    0,    0],
       [ 188,    9,  202, ...,    0,    0,    0],
       [7636, 7656,   48, ...,    0,    0,    0]], dtype=int32)

In [ ]:
embedding_dim = 200
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=max_seq_length))
#model.add(LSTM(units=100, dropout=0.5, recurrent_dropout=0.1, return_sequences=True))
#model.add(LSTM(units=150, dropout=0.5, recurrent_dropout=0.2, return_sequences=True))
#model.add(LSTM(units=200, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
batch_size = 1024
epochs = 5
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train_pad, train_data['Label'], batch_size=batch_size, epochs=epochs, validation_data=(X_val_pad, val_data['Label']),callbacks=[early_stop])

Epoch 1/5
88/88 [==============================] - 16s 115ms/step - loss: 0.6920 - accuracy: 0.5274 - val_loss: 0.6901 - val_accuracy: 0.5462
Epoch 2/5
88/88 [==============================] - 10s 108ms/step - loss: 0.6897 - accuracy: 0.5404 - val_loss: 0.6885 - val_accuracy: 0.5451
Epoch 3/5
88/88 [==============================] - 9s 107ms/step - loss: 0.6888 - accuracy: 0.5396 - val_loss: 0.6878 - val_accuracy: 0.5449
Epoch 4/5
88/88 [==============================] - 9s 102ms/step - loss: 0.6885 - accuracy: 0.5394 - val_loss: 0.6875 - val_accuracy: 0.5449
Epoch 5/5
88/88 [==============================] - 9s 106ms/step - loss: 0.6883 - accuracy: 0.5396 - val_loss: 0.6874 - val_accuracy: 0.5450


In [ ]:
y_pred = model.predict(X_val_pad)


1195/1195 [==============================] - 2s 2ms/step


In [ ]:
y_pred_labels = np.where(y_pred > 0.5, 1, 0)

In [ ]:
import numpy as np

unique_labels_val = np.unique(val_data['Label'])
unique_labels_pred = np.unique(y_pred_labels)

print("Unique labels in val_data['Label']: ", unique_labels_val)
print("Unique labels in y_pred_labels: ", unique_labels_pred)



Unique labels in val_data['Label']:  [0 1]
Unique labels in y_pred_labels:  [0 1]


In [ ]:
print(val_data['Label'].unique())

[1 0]


In [ ]:
filtered_val_data = val_data[val_data['Label'].isin([0, 1])]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

true_labels = filtered_val_data['Label']
precision = precision_score(true_labels, y_pred_labels)
recall = recall_score(true_labels, y_pred_labels)
f1 = f1_score(true_labels, y_pred_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

ValueError: ignored

In [ ]:
print(classification_report(filtered_val_data['Label'], y_pred_labels))


ValueError: ignored

In [ ]:

from sklearn.metrics import classification_report

# Convert the target labels to integers
y_val_int = val_data['Label'].astype(int)

# Convert the predicted labels to integers
y_pred_int = y_pred_labels.astype(int)

# Ensure the target and predicted labels have the same shape
y_val_int = y_val_int[:len(y_pred_int)]

# Generate the classification report
print(classification_report(y_val_int, y_pred_int))


ValueError: ignored

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(val_data['Label'], y_pred_labels))

ValueError: ignored

In [ ]:
from yellowbrick.classifier import ClassificationReport

visualizer = ClassificationReport(model, classes=['FALSE', 'TRUE'])
visualizer.fit(X_train_pad, train_data['Label'])
visualizer.score(X_val_pad, val_data['Label'])
visualizer.show()

YellowbrickTypeError: ignored

In [ ]:
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve
cv = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

lc_viz = LearningCurve(
    model, cv=cv, scoring='f1_weighted', n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

lc_viz.fit(X_train_pad, train_data['Label'])
lc_viz.show()